# Automação Web e Busca de Informações com Python

In [9]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
#abrir um navegador
navegador=webdriver.Chrome()
#ir p/ site novegador
navegador.get('https://www.google.com')
#pegar cotaçao dólar
navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação dólar')
navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_dolar= navegador.find_element(By.XPATH,'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
#pegar cotação euro
navegador.get('https://www.google.com')
navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação euro')
navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_euro= navegador.find_element(By.XPATH,'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
#pegar cotação ouro
navegador.get('https://www.melhorcambio.com/ouro-hoje#')
cotacao_ouro= navegador.find_element(By.XPATH,'//*[@id="comercial"]').get_attribute('value')
cotacao_ouro= cotacao_ouro.replace(',', '.')
print(cotacao_dolar)
print(cotacao_euro)
print(cotacao_ouro)
navegador.quit()

4.7968
5.17322888
298.54


### Agora vamos atualizar a nossa base de preços com as novas cotações

- Importando a base de dados

In [10]:
#passo 4: importar a lista de produtos
import pandas as pd

tabela= pd.read_excel('Produtos.xlsx')
display(tabela)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [11]:
#Passo 5: Recalcular o preço de cada produto
#atualizar a cotação
tabela.loc[tabela['Moeda']== 'Dólar', 'Cotação'] = float(cotacao_dolar)
tabela.loc[tabela['Moeda']== 'Euro', 'Cotação'] = float(cotacao_euro)
tabela.loc[tabela['Moeda']== 'Ouro', 'Cotação'] = float(cotacao_ouro)
#atualizar o preço base reais
tabela['Preço Base Reais']= tabela['Preço Base Original'] * tabela['Cotação']
#atualizar o preço final
tabela['Preço Final']= tabela['Preço Base Reais'] * tabela['Margem']
display(tabela)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,4.796800,4796.752032,1.40,6715.452845
1,Carro Renault,4500.00,Euro,5.173229,23279.529960,2.00,46559.059920
2,Notebook Dell,899.99,Dólar,4.796800,4317.072032,1.70,7339.022454
3,IPhone,799.00,Dólar,4.796800,3832.643200,1.70,6515.493440
4,Carro Fiat,3000.00,Euro,5.173229,15519.686640,1.90,29487.404616
5,Celular Xiaomi,480.48,Dólar,4.796800,2304.766464,2.00,4609.532928
6,Joia 20g,20.00,Ouro,298.540000,5970.800000,1.15,6866.420000


### Agora vamos exportar a nova base de preços atualizada

In [12]:
#Passo 6: Salvar os novos preços do produto
tabela.to_excel('Produtos novo.xlsx', index=False)